# Pronóstico automatizado de productos RDS (01.01.2024-31.05.2025) vendidos por seller Repuestosdelsol

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
import os
from pipeline.procesamiento_bases import procesar_por_par_producto_marca, procesar_para_powerBI
from pipeline.modelos_pronosticos_ventas import crear_lags, ejecutar_auto_arima, ejecutar_xgboost, ejecutar_prophet

In [ ]:
path_df_2024 = '/Users/martincarrasco/Desktop/Martín_Carrasco/Reportes/2024/Cuentas RDS/Ventas Consolidado RDS 2024.csv'
path_df_2025 = '/Users/martincarrasco/Desktop/Martín_Carrasco/Reportes/2025/Cuentas RDS/Ventas Consolidado RDS 2025 (hasta 22-06-2025).csv'
fecha_limite = "2025-06-01"
seller = 'REPUESTOS DEL SOL'
safe_seller = seller.replace(' ', '_')
carpeta_graficos_autoarima = f'/Users/martincarrasco/Desktop/Martín_Carrasco/Reportes/Interanuales/Cuentas RDS/Pronósticos/Pred_{safe_seller}_ene25_may_25_autoarima_productomarca'
carpeta_graficos_xgboost = f'/Users/martincarrasco/Desktop/Martín_Carrasco/Reportes/Interanuales/Cuentas RDS/Pronósticos/Pred_{safe_seller}_ene25_may_25_xgboost_productomarca'
carpeta_graficos_prophet = f'/Users/martincarrasco/Desktop/Martín_Carrasco/Reportes/Interanuales/Cuentas RDS/Pronósticos/Pred_{safe_seller}_ene25_may_25_prophet_productomarca'

In [ ]:
df_2024 = procesar_por_par_producto_marca(path_df_2024, fecha_limite, año = 2024, seller = seller)
df_2024

/Users/martincarrasco/Desktop/Martín_Carrasco/pipeline/pipeline/procesamiento_bases.py:15: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path_csv, skiprows = header_line, sep = sep)
/Users/martincarrasco/Desktop/Martín_Carrasco/pipeline/pipeline/procesamiento_bases.py:74: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fechas_completas = pd.date_range(start=df['Fecha_Compra'].min(),


,Producto,Marca,Fecha_Compra,Unidades_Vendidas
0,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-01-31,0
1,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-02-29,0
2,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-03-31,0
3,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-04-30,0
4,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-05-31,0
...,...,...,...,...
168727,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2024-08-31,0
168728,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2024-09-30,0
168729,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2024-10-31,0
168730,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2024-11-30,0


In [ ]:
df_2025 = procesar_por_par_producto_marca(path_df_2025, fecha_limite, año = 2025, seller = seller)
df_2025

/Users/martincarrasco/Desktop/Martín_Carrasco/pipeline/pipeline/procesamiento_bases.py:74: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  fechas_completas = pd.date_range(start=df['Fecha_Compra'].min(),


,Producto,Marca,Fecha_Compra,Unidades_Vendidas
0,Alternador Peugeot 405gl 92/98,GENÉRICO,2025-01-31,0
1,Alternador Peugeot 405gl 92/98,GENÉRICO,2025-02-28,1
2,Alternador Peugeot 405gl 92/98,GENÉRICO,2025-03-31,0
3,Alternador Peugeot 405gl 92/98,GENÉRICO,2025-04-30,0
4,Alternador Peugeot 405gl 92/98,GENÉRICO,2025-05-31,0
...,...,...,...,...
44010,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-01-31,1
44011,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-02-28,0
44012,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-03-31,0
44013,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-04-30,0


In [ ]:
df_2024_2025 = pd.concat([df_2024, df_2025], ignore_index = True)
df_2024_2025

,Producto,Marca,Fecha_Compra,Unidades_Vendidas
0,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-01-31,0
1,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-02-29,0
2,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-03-31,0
3,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-04-30,0
4,Abrazadera Barra Estab Para Hyundai Tucs,HYUNDAI,2024-05-31,0
...,...,...,...,...
212742,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-01-31,1
212743,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-02-28,0
212744,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-03-31,0
212745,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-04-30,0


## Modelo ARIMA con auto_arima

In [ ]:
resultados_arima, predicciones_arima = ejecutar_auto_arima(df = df_2024_2025, columnas_grupo = ['Producto', 'Marca'], carpeta_graficos = carpeta_graficos_autoarima, seller = seller)

In [ ]:
resultados_arima

In [ ]:
predicciones_arima

## Modelo XGBoost

In [ ]:
resultados_xgb, predicciones_xgb = ejecutar_xgboost(df = df_2024_2025, columnas_grupo = ['Producto', 'Marca'], carpeta_graficos = carpeta_graficos, seller = seller)

In [7]:
resultados_xgb

,Producto,Marca,MAE,RMSE
0,Alternador Chevrolet Luv Dmax 3.0 2004 2,DENNO,2.315043,3.027048
1,Alternador Chevrolet Spark 800 2006 2016,GENÉRICO,0.233793,0.387447
2,Alternador Chevrolet Vivant 1.6 Mini Van,GENÉRICO,0.250213,0.403143
3,"Alternador Hyundai Tucson 2004 2010 2,0,",GENÉRICO,0.599202,0.705906
4,Alternador Jeep Compass 2.4 2006 2017,ECHLIN,0.666667,1.010218
...,...,...,...,...
3804,Vidrio Espejo Izquierdo Para Hyundai Acc,HYUNDAI,0.200861,0.380260
3805,Volante Bimasa Para Kia Carens - 2007 /,KIA,0.200861,0.380260
3806,X2 Amortiguador Delantero Chevrolet Optr,GENÉRICO,6.413038,6.988499
3807,X2 Farol Trasero Nissan Np300 2 Unidade,GENÉRICO,0.299904,0.499783


In [8]:
predicciones_xgb

,Producto,Marca,Fecha_Compra,Real,Pred
0,Alternador Chevrolet Luv Dmax 3.0 2004 2,DENNO,2025-01-31,0,0.999988
1,Alternador Chevrolet Luv Dmax 3.0 2004 2,DENNO,2025-02-28,6,0.001614
2,Alternador Chevrolet Luv Dmax 3.0 2004 2,DENNO,2025-03-31,0,0.418318
3,Alternador Chevrolet Luv Dmax 3.0 2004 2,DENNO,2025-04-30,0,1.999104
4,Alternador Chevrolet Luv Dmax 3.0 2004 2,DENNO,2025-05-31,3,0.840579
...,...,...,...,...,...
19040,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-01-31,1,0.000020
19041,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-02-28,0,0.000020
19042,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-03-31,0,0.000020
19043,pticos Tuning Para Nissan V16 2003 2011,DIFORZA,2025-04-30,0,0.000020


## Modelo Prophet

In [ ]:
resultados_prophet, predicciones_prophet = ejecutar_prophet(df = df_2024_2025, columnas_grupo = ['Producto', 'Marca'], carpeta_graficos = carpeta_graficos_prophet, seller = seller)

In [ ]:
resultados_prophet

In [ ]:
predicciones_prophet